In [1]:
import pandas as pd
import numpy as np
import pickle
import random
import os

In [2]:
root = '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/'

'''
Context: 
group_id,day_of_week,time_of_day,dosage,location,weather,previous_step_count,last_step_count,variation_in_step_counts
'''

In [3]:
with open('{}step_dist_both_groups.pkl'.format(root),'rb') as f:
    dists = pickle.load(f)

In [4]:
with open('{}interventions_both_groups.pkl'.format(root),'rb') as f:
    intervention_dists = pickle.load(f)

In [6]:
days = pd.date_range(start = '7/22/2015',end =pd.Timestamp('7/22/2015')+pd.DateOffset(days=90),freq='60T')

In [25]:
def get_location_prior(group_id,day_of_week,time_of_day):
    with open('{}initial_location_distributions.pkl'.format(root),'rb') as f:
        loc_lookup = pickle.load(f)
    key = '{}-{}-{}'.format(group_id,day_of_week,time_of_day)
    
    ##make a bit smoother while loop instead 
    if key in loc_lookup:
        ps = loc_lookup[key]
    else:
        key =  '{}-{}'.format(group_id,day_of_week)
        if key  in loc_lookup:
            ps = loc_lookup[key]
        else:
            key =  '{}'.format(group_id)
            if key  in loc_lookup:
                ps = loc_lookup[key]
                
            else:
                ps = loc_lookup['mean']
                
    val = np.argmax(np.random.multinomial(100,ps))
    return val

In [26]:
def get_weather_prior(group_id,day_of_week,time_of_day):
    with open('{}initial_weather_distributions.pkl'.format(root),'rb') as f:
        loc_lookup = pickle.load(f)
    key = '{}-{}-{}'.format(group_id,day_of_week,time_of_day)
    
    ##make a bit smoother while loop instead 
    if key in loc_lookup:
        ps = loc_lookup[key]
    else:
        key =  '{}-{}'.format(group_id,day_of_week)
        if key  in loc_lookup:
            ps = loc_lookup[key]
        else:
            key =  '{}'.format(group_id)
            if key  in loc_lookup:
                ps = loc_lookup[key]
                
            else:
                ps = loc_lookup['mean']
                
    val = np.argmax(np.random.multinomial(100,ps))
    return val

In [27]:
def get_time_of_day(an_index):
    with open('{}hour_to_id.pkl'.format(root),'rb') as f:
        hour_lookup = pickle.load(f)
    return hour_lookup[an_index.hour]
    

In [28]:
def get_day_of_week(an_index):
    with open('{}day_to_id.pkl'.format(root),'rb') as f:
        hour_lookup = pickle.load(f)
    return hour_lookup[an_index.dayofweek]
    

In [55]:
def get_index(key):
    
    keys = ['group_id','day_of_week','time_of_day','dosage','location','weather','previous_step_count','last_step_count','variation']
    
    kl = {keys[i]:i for i in range(len(keys))}
    
    return kl[key]
    

In [50]:
def get_initial_context(num_people,first_index):
    '''States:
    [group,day_of_week,time_of_day,location,weather,dosage,previous_step_count]
    
    '''
    
    all_people = []
    for person in range(num_people):
        #group_id = int(random.random()>.8)+1
        group_id = 2
        day_of_week = get_time_of_day(first_index)
        time_of_day = get_day_of_week(first_index)
        first_location = get_location_prior(group_id,day_of_week,time_of_day)
        weather = get_weather_prior(group_id,day_of_week,time_of_day)
        #weather = 0 
        dosage = 1
        variation = 1
        previous_step_count = 0 
        lsc = 0
        #day_of_week,time_of_day,dosage,location,last_steps,last_steps_hour,varia
        all_people.append([group_id,day_of_week,time_of_day,dosage,first_location,weather,previous_step_count,lsc,variation])
        
    return all_people

In [30]:
def get_initial_steps(contexts):
    
    return [steps_given_context(person_context) for person_context in contexts]

    

In [31]:
def get_time(current_time):
    
    #needs to be a time delta
    
    return current_time+1
    

In [32]:
def get_possible_keys(context):
    
    
    keys = []
    
    
    for i in range(len(context)):
        stop = len(context)-i-1
        #for j in range(stop):
        if stop>=1:
            key = '-'.join([str(context[j]) for j in range(stop)])
        
        keys.append(key)
    keys.append('{}-mean'.format(context[0]))
    return keys

In [33]:
def get_steps_no_action(context):
    context_key = '-'.join([str(c) for c in context])
    possible_keys = get_possible_keys(context)
    keys = [context_key]
    keys.extend(possible_keys)
    #print(keys)
    #keys = [k+'-{}'.format(action) for k in keys]
    #print(keys)
    i=0
    while keys[i] not in dists:
        #print(i)
        i=i+1
    #print(keys[i])
    #print(keys[-1])
    dist = dists[keys[i]]
    
    
    x = np.random.normal(loc=dist[0],scale=dist[1])
    while(x<0):
         x = np.random.normal(loc=dist[0],scale=dist[1])
    return x
    

In [34]:
def get_steps_action(context,action):
    #nkey = '{}-{}'.format(action,context)

    this_context = [action]
    this_context.extend(context)
    possible_keys = get_possible_keys(this_context)
    
    context_key = '-'.join([str(c) for c in this_context])
    
    keys = [context_key]
    keys.extend(possible_keys)
    #print(keys)
    #keys = [k+'-{}'.format(action) for k in keys]
    #print(keys)
    i=0
    while keys[i] not in intervention_dists:
        #print(i)
        i=i+1
    #print(keys[i])
    #print(keys[-1])
    dist = intervention_dists[keys[i]]
    
    
    x = np.random.normal(loc=dist[0],scale=dist[1])
    while(x<0):
         x = np.random.normal(loc=dist[0],scale=dist[1])
    return x
    

In [35]:
def get_steps(context,action):
    if action==-1:
        return get_steps_no_action(context)

    return get_steps_action(context,action)

In [63]:
def get_next_location(context):
    
    with open('{}location_conditiononed_on_last_location.pkl'.format(root),'rb') as f:
        loc_dists =pickle.load(f)
    
    context_key = '-'.join([str(c) for c in context])
    possible_keys = get_possible_keys(context)
    
    keys = [context_key]
    keys.extend(possible_keys)
    #print(possible_keys)
    i=0
    #print(keys[-1])
    while keys[i] not in loc_dists and i<len(keys):
        i=i+1
    dist = loc_dists[keys[i]]
    
    val = np.argmax(np.random.multinomial(100,dist))
    
    return val
            
                
    
    
    

In [74]:
def get_next_weather(context):
    
    with open('{}weather_conditiononed_on_last_weather.pkl'.format(root),'rb') as f:
        loc_dists =pickle.load(f)
    
    relevant_context = [context[get_index('time_of_day')],context[get_index('weather')]]
    
    context_key = '-'.join([str(c) for c in relevant_context])
    possible_keys = get_possible_keys(relevant_context)
    
    keys = [context_key]
    keys.extend(possible_keys)
    #print(keys)
    i=0
    #print(keys[-1])
    while keys[i] not in loc_dists and i<len(keys):
        i=i+1
    dist = loc_dists[keys[i]]
    
    val = np.argmax(np.random.multinomial(100,dist))
    
    return val

In [37]:
def get_new_previous_step_count(some_steps):
    ##pull in distribution threshold lookup thing
    return 0
    

In [38]:
##what do I need here
def get_new_dosage(current_dosage,action):
    if action==1:
        current_dosage = current_dosage+2
    else:
        current_dosage=current_dosage-1
    if current_dosage>100:
        current_dosage=100
    if current_dosage<1:
        current_dosage=1 
    return current_dosage

In [66]:
def get_context_revised(current_index,current_context,current_steps,decision_time,lsc,variation,last_action):
        
    day_of_week = get_time_of_day(current_index)
    time_of_day = get_day_of_week(current_index)
    
    
    new_previous_step_count = get_new_previous_step_count(current_steps)
    
    
    if decision_time:
        location = get_next_location([current_context[get_index('group_id')],day_of_week,time_of_day,current_context[get_index('location')]])
    
        dosage = get_new_dosage(current_context[get_index('dosage')],last_action)
        
        weather = get_next_weather(current_context)
        
    else:
        location = current_context[get_index('location')]
        dosage = current_context[get_index('dosage')]
        weather = current_context[get_index('weather')]
        
        
        
    return [current_context[0],day_of_week,time_of_day,dosage,location,weather,new_previous_step_count,lsc,variation]
    
    

In [53]:
def get_new_lsc(step_slice):
    #print('hi there')
    s =sum(step_slice)**.5
    if s<0:
        return 0
    if s>203:
        return 203
    return s

In [54]:
def get_variation_pre_week(variation,all_steps,time_indices,i):
    
    two_days = time_indices[0].date()+pd.DateOffset(days=1)
    is_set = False
    #first_index_last_day=-1
    if i>two_days:
        #print(i)
        for j in range(len(time_indices)):
            if time_indices[j].date()==i.date():
                if not is_set:
                    #print('set')
                    #print(j)
                    first_index_last_day = j
                    #print(first_index_last_day)
                    is_set = True
            if time_indices[j]== i:
                last_index_last_day = j
    
        pre_steps = all_steps[:first_index_last_day]
        post_steps = all_steps[first_index_last_day:last_index_last_day]
        
        #print(pre_steps)
        #print(post_steps)
        
        return int(np.array(pre_steps).std()>np.array(post_steps).std())
        
    else:
        return variation
    
    

In [42]:
#will be algorithm, needs to communicate with algorithm
def get_action(initial_context,steps):
    return 1

In [57]:
def simulate_run(num_people,time_indices,decision_times):
    
    
    initial_context = get_initial_context(num_people,time_indices[0])
    
    
    initial_context = initial_context[0]
    
    
    
    
    initial_steps = get_steps(initial_context,0)
    current_steps = initial_steps
    action = -1 
    all_steps = []
    
    last_day = time_indices[0]
    
    new_day = False
    
    #for d in range(num_days):
    
    
    start_of_day = 0 
    end_of_day=0
    current_index=0
    
    
    first_week = time_indices[0].date()+pd.DateOffset(days=7)
    
    for i in time_indices:
        
        if i.date()!=last_day.date():
            #print('trigger')
            #print(i.date())
            #print(last_day.date())
            #print('hi there')
            new_day=True
            
            
            
        decision_time = bool(i in decision_times)
        #print(decision_time)
        if i!=time_indices[0]:
            #decision_time = bool(i in decision_times)
            
            ##need to modify this
            #my_context = get_context(initial_context,current_steps,i,decision_time)
            lsc = initial_context[get_index('last_step_count')]
            variation = initial_context[get_index('variation')]
            if new_day:
                #lsc=0
                
                
                ##would love to break this out more cleanly 
                if i<first_week:
                    variation = get_variation_pre_week(variation,all_steps,time_indices,last_day)
                else:
                    variation = get_variation(all_steps,time_indices,last_day)
                
                
                lsc = get_new_lsc(all_steps[start_of_day:end_of_day])
                #variation = get_new_variation()
                
            
            ##action will be the last action
            my_context = get_context_revised(i,initial_context,current_steps,decision_time,lsc,variation,action)
            #return my_context
            if i in decision_times:
                print('decision time')
                action = get_action(my_context,current_steps)
                print(action)
            else:
                action = -1
            ##redo get_steps
            next_steps = get_steps(my_context,action) 
            all_steps.append(next_steps)
            initial_context = my_context
            current_steps = next_steps
        else:
            if i in decision_times:
                print('decision time')
                action = get_action(initial_context,current_steps)
            else:
                action = -1
            next_steps = get_steps(initial_context,action) 
            all_steps.append(next_steps)
            current_steps = next_steps
        if new_day:
            
            start_of_day = current_index
            new_day=False
        last_day =i
        end_of_day = current_index  
        current_index = current_index+1
            
    return all_steps

In [58]:
decision_times = ['2015-07-22 02:00:00']
num_people = 2

In [59]:
days = pd.date_range(start = '7/22/2015',end =pd.Timestamp('7/22/2015')+pd.DateOffset(days=90),freq='60T')

In [60]:
temp_indices = days[:24*7]


In [75]:
x=simulate_run(num_people,temp_indices,pd.DatetimeIndex(['2015-07-28 14:00:00']))

decision time
1


In [76]:
x

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 805.3259052745814,
 874.3717991837161,
 423.3893687461939,
 286.7484467433249,
 2124.2928079897047,
 1030.267338186014,
 26.284584621289923,
 758.0642610183802,
 827.3600024610912,
 70.1674868389282,
 1347.9586793341357,
 149.9775632701743,
 2029.3455462683091,
 148.0185676938812,
 986.0558136510215,
 1665.7370758517736,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1036.0357436850707,
 340.1070113372219,
 1811.435718873815,
 717.8758097769681,
 142.29906881515114,
 1240.011667641013,
 3249.9233212705376,
 743.8272301951129,
 1813.5179040700968,
 774.8769946068205,
 1607.4774912610633,
 986.5248529389015,
 211.20764356494237,
 2089.2176325391515,
 256.88602995788017,
 1480.7867079979137,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1366.4322970451512,
 221.9846970811838,
 451.0682656855455,
 48.65280227498124,
 863.0486806074165,
 745.2452945482405,
 409.968462690142,
 1021.8139642593857,
 642.8307036064859,
 100.85897788447869,
 

In [291]:
all_runs = []
for i in range(500):
    all_runs.append(simulate_run(num_people,temp_indices,decision_times))